In [ ]:
!pip install -q torchaudio
!pip install -q transformers
!pip install -q datasets
!pip install -q lang_trans
!pip install -q arabic_reshaper
!pip install -q python-bidi
!pip install -q pydub
!pip install -q soundfile
!pip install -q jiwer
!pip install -q PyArabic

     |████████████████████████████████| 4.9 MB 38.8 MB/s 
     |████████████████████████████████| 120 kB 72.8 MB/s 
     |████████████████████████████████| 6.6 MB 76.6 MB/s 
     |████████████████████████████████| 431 kB 33.6 MB/s 
     |████████████████████████████████| 212 kB 76.8 MB/s 
     |████████████████████████████████| 115 kB 92.5 MB/s 
     |████████████████████████████████| 127 kB 75.6 MB/s 
     |████████████████████████████████| 1.4 MB 46.9 MB/s 
     |████████████████████████████████| 1.6 MB 60.6 MB/s 
     |████████████████████████████████| 104 kB 67.9 MB/s 
     |████████████████████████████████| 126 kB 27.4 MB/s 


# Imports

In [ ]:
import transformers
transformers.utils.move_cache()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
import tensorflow as tf

In [ ]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import transformers
import pandas as pd
from torch import nn
import seaborn as sns
import torch.nn as nn
from tqdm import tqdm
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from packaging import version
from pydub import AudioSegment
from dataclasses import asdict
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser 
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from typing import Any, Dict, List, Optional, Union
from sklearn.model_selection import train_test_split
from transformers import HfArgumentParser,TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import is_apex_available,set_seed ,Trainer,Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel
%matplotlib inline

# Helper classes

## CTC Trainer

In [ ]:
class CTCTrainer(Trainer):
    def training_step(self, model, inputs):

        model.train()
        inputs = self._prepare_inputs(inputs)
        loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        loss.backward()

        return loss.detach()

    def compute_loss(self, model, inputs, return_outputs=False):
        # labels = inputs.pop("labels").to('cuda')
        labels = inputs["labels"].to("cuda")
        outputs = model(**inputs)
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(outputs["logits"], labels.argmax(-1).long())
        return (loss, outputs) if return_outputs else loss

## Processors

In [ ]:
class CustomWav2Vec2Processor:
    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor
        self.current_processor = self.feature_extractor

    def save_pretrained(self, save_directory):
        self.feature_extractor.save_pretrained(save_directory)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, **kwargs):
        feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
            pretrained_model_name_or_path,
            **kwargs,
        )
        return cls(feature_extractor=feature_extractor)

    def __call__(self, *args, **kwargs):
        return self.current_processor(*args, **kwargs)

    def pad(self, *args, **kwargs):
        return self.current_processor.pad(*args, **kwargs)

## Model Args

In [ ]:
def list_field(default=None, metadata=None):
    return field(default_factory=lambda: default, metadata=metadata)

@dataclass
class ModelArguments:

    model_name_or_path: str = field(
        metadata={
            "help": "Path to pretrained model or model identifier from huggingface.co/models"
        }
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={
            "help": "Where do you want to store the pretrained models downloaded from huggingface.co"
        },
    )
    freeze_feature_extractor: Optional[bool] = field(
        default=True,
        metadata={
            "help": "Whether to freeze the feature extractor layers of the model."
        },
    )
    attention_dropout: Optional[float] = field(
        default=0.1,
        metadata={"help": "The dropout ratio for the attention probabilities."},
    )
    activation_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout ratio for activations inside the fully connected layer."
        },
    )
    hidden_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler."
        },
    )
    feat_proj_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout probabilitiy for all 1D convolutional layers in feature extractor."
        },
    )
    mask_time_prob: Optional[float] = field(
        default=0.05,
        metadata={
            "help": "Propability of each feature vector along the time axis to be chosen as the start of the vector"
            "span to be masked. Approximately ``mask_time_prob * sequence_length // mask_time_length`` feature"
            "vectors will be masked along the time axis. This is only relevant if ``apply_spec_augment is True``."
        },
    )
    gradient_checkpointing: Optional[bool] = field(
        default=True,
        metadata={
            "help": "If True, use gradient checkpointing to save memory at the expense of slower backward pass."
        },
    )
    layerdrop: Optional[float] = field(
        default=0.0, metadata={"help": "The LayerDrop probability."}
    )


@dataclass
class DataTrainingArguments:

    dataset_config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "The configuration name of the dataset to use (via the datasets library)."
        },
    )
    train_split_name: Optional[str] = field(
        default="train+validation",
        metadata={
            "help": "The name of the training data set split to use (via the datasets library). Defaults to 'train'"
        },
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached preprocessed datasets or not."},
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_val_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of validation examples to this "
            "value if set."
        },
    )
    chars_to_ignore: List[str] = list_field(
        default=[",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�"],
        metadata={"help": "A list of characters to remove from the transcripts."},
    )


@dataclass
class DataCollatorCTCWithPadding:

    processor: CustomWav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = 320000
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = 320000
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods

        input_features = [
            {"input_values": feature["input_values"]} for feature in features
        ]

        def onehot(lbl):
            onehot = [0] * 16
            onehot[int(lbl)] = 1
            return onehot

        output_features = [onehot(feature["labels"]) for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=True,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # for val in batch['input_values']:
        #   print(val[:10])
        #   print(val[-10:])
        # print(batch['input_values'].shape)
        batch["labels"] = torch.tensor(output_features)
        # print(batch["labels"].argmax(-1))
        return batch

## Model

In [ ]:
class Wav2Vec2ClassificationModel(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.wav2vec2 = Wav2Vec2Model(config)

        #https://www.youtube.com/watch?v=0_PgWWmauHk&t=81s&ab_channel=PythonEngineer

        # self.inner_dim = 128
        self.feature_size = 999
        self.input_size = 1024
        self.hidden_size = 256
        self.num_layers=4

        # self.tanh = nn.Tanh()
        # self.linear1 = nn.Linear(1024, self.inner_dim)
        # self.linear2 = nn.Linear(self.inner_dim * self.feature_size, 16)
        self.BiGRU = nn.GRU(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            bidirectional=True,
            batch_first=True
          )
        self.linear = nn.Linear(self.hidden_size*2, 16)
        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def forward(
        self,
        input_values,
        attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None,
    ):
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        x = outputs[0]
        h0 = torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(self.device)
        x,_ = self.BiGRU(x,h0)
        x = x[:,-1,:] # get the hidden state of the last timestep
        x = self.linear(x)
        return {"logits": x}

# Getting the dataset

In [ ]:
# https://superuser.com/a/118783/1046355
!cp -r -u -p /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/dataset_wav.zip .

In [ ]:
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx .
!cp /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx .

In [ ]:
if os.path.exists('dataset_wav'):
  if len(os.listdir('dataset_wav')) == 0:
    os.system('unzip dataset_wav.zip')
else:
  os.system('unzip dataset_wav.zip')

In [ ]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx'
test_metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx'

In [ ]:
dataset_wav_folder = 'dataset_wav'

In [ ]:
def preprocess_metadata(metadata):
  metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
  metadata['Clean Script'] = metadata['Script'].apply(lambda script: araby.strip_tashkeel(araby.strip_tatweel(script)))
  metadata['File Exist'] = metadata['Utterance name'].apply(lambda file_name: os.path.isfile(f'{dataset_wav_folder}/{file_name}'))
  metadata = metadata[metadata['File Exist'] == True]
  return metadata

In [ ]:
metadata = preprocess_metadata(pd.read_excel(metadata_path))
test_metadata = preprocess_metadata(pd.read_excel(test_metadata_path))
train_metadata = metadata[[script not in test_metadata['No.'].values for script in metadata['No.']]]
train_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Is_Malformed,Reciter_Label,Reciter_Gender,Further info on Bahr,Name of document,Clean Script,File Exist
0,1.0,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,0.0,0.0,0.0,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع ال...,True
1,2.0,P-2.aac,حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ من...,المديد,ديوان الطرماح ص227,2.aac,2.aac,0.0,0.0,0.0,NaN,NaN,حسرت عنه الرياح فأبدت منتأى كالقرو رهن ا...,True
2,3.0,P-3.aac,وخصيفَ اللَّونِ جادَتْ بهِ مَرْخَةٌ مِنْ ...,المديد,ديوان الطرماح ص227,3.aac,3.aac,0.0,0.0,0.0,NaN,NaN,وخصيف اللون جادت به مرخة من مخدج أو تمام,True
3,4.0,P-4.aac,بَيْنَ أظْآرٍ بِمَظْلُومَةٍ كَسَرَاةِ الس...,المديد,ديوان الطرماح ص227,4.aac,4.aac,0.0,0.0,0.0,NaN,NaN,بين أظآر بمظلومة كسراة الساق ساق الحمام,True
4,5.0,P-5.aac,مَنْزِلًا كَانَ لَنَا مَرَّةً وطنًا نحتلّ...,المديد,ديوان الطرماح ص227,5.aac,5.aac,0.0,0.0,0.0,NaN,NaN,منزلا كان لنا مرة وطنا نحتله كل عام,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,3731.0,P-3731.mp3,وَلِّ السَلاطينَ مَن تَوَلّاها وَاِلجَأ إ...,المنسرح,-,45.mp3,45.mp3,NaN,NaN,NaN,NaN,NaN,ول السلاطين من تولاها والجأ إليه تكن حدياها,True
3682,3732.0,P-3732.mp3,وَلا تَغُرَّنَّكَ الإِمارَةُ في غَيرِ أَم...,المنسرح,-,46.mp3,46.mp3,NaN,NaN,NaN,NaN,NaN,ولا تغرنك الإمارة في غير أمير وإن بها باهى,True
3683,3733.0,P-3733.mp3,فَإِنَّما المَلكُ رَبُّ مَملَكَةٍ قَد فَغ...,المنسرح,-,47.mp3,47.mp3,NaN,NaN,NaN,NaN,NaN,فإنما الملك رب مملكة قد فغم الخافقين سرياها,True
3684,3734.0,P-3734.mp3,مُبتَسِمٌ وَالوُجوهُ عابِسَةٌ سِلمُ العِد...,المنسرح,-,48.mp3,48.mp3,NaN,NaN,NaN,NaN,NaN,مبتسم والوجوه عابسة سلم العدى عنده كهيجاها,True


In [ ]:
test_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,89.0,P-89.aac,إنَّ بالشِّعبِ الَّذي دُونَ سَلْعٍ لَقتيل...,المديد,ديوان ثابت بن جابرص35,89.aac,89.aac,NaN,NaN,إن بالشعب الذي دون سلع لقتيلا دمه ما يطل,True
1,90.0,P-90.aac,خَلَّفَ الْعِبْءَ عَلَيَّ وَولَّى أنا بال...,المديد,ديوان ثابت بن جابرص35,90.aac,90.aac,NaN,NaN,خلف العبء علي وولى أنا بالعبء له مستقل,True
2,91.0,P-91.aac,ووراءَ الثَّأر منِّي ابنُ أختٍ مَصِعٌ عُق...,المديد,ديوان ثابت بن جابرص35,91.aac,91.aac,NaN,NaN,ووراء الثأر مني ابن أخت مصع عقدته ما تحل,True
3,92.0,P-92.aac,مُطْرِقٌ يَرْشَحُ سَمًّا كَمَا أَطْ رَقَ ...,المديد,ديوان ثابت بن جابرص35,92.aac,92.aac,NaN,NaN,مطرق يرشح سما كما أط رق أفعى ينفث السم صل,True
4,93.0,P-93.aac,خبرٌ ما نابَنا مُصْمَئِلٌّ جلَّ حتّى دقَّ...,المديد,ديوان ثابت بن جابرص35,93.aac,93.aac,NaN,NaN,خبر ما نابنا مصمئل جل حتى دق فيه الأجل,True
...,...,...,...,...,...,...,...,...,...,...,...
354,3682.0,P-3682.m4a,حِسانُ الوُجوهِ حِدادُ السُيو فِ يَبتَدِر...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (98).m4a,التسجيل (98).m4a,NaN,NaN,حسان الوجوه حداد السيو ف يبتدر المجد شبانها,True
355,3683.0,P-3683.m4a,وَبِالشَوطِ مِن يَثرِبٍ أَعبُدٌ سَتَهلِكُ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (99).m4a,التسجيل (99).m4a,NaN,NaN,وبالشوط من يثرب أعبد ستهلك في الخمر أثمانها,True
356,3684.0,P-3684.m4a,يَهونُ عَلى الأَوسِ أَثمانُهُم إِذا راحَ ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (100).m4a,التسجيل (100).m4a,NaN,NaN,يهون على الأوس أثمانهم إذا راح يخطر نشوانها,True
357,3685.0,P-3685.m4a,أَتَتهُم عَرانينُ مِن مالِكٍ سِراعٌ إِلى ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (101).m4a,التسجيل (101).m4a,NaN,NaN,أتتهم عرانين من مالك سراع إلى الروع فتيانها,True


In [ ]:
train_metadata,val_metadata = train_test_split(train_metadata, test_size=0.1,random_state=45)

# Training args

In [ ]:
OUTPUT_PATH = '/content/drive/MyDrive/KFUPM-Master/ICS606/PostCourseWork/Models/EndToEndBiGRUModel'
args_string=f"""
--model_name_or_path="bakrianoo/sinai-voice-ar-stt"
--output_dir={OUTPUT_PATH}
--overwrite_output_dir=1
--freeze_feature_extractor 
--num_train_epochs="5" 
--per_device_train_batch_size="2"
--learning_rate="3e-5" 
--warmup_steps="5" 
--evaluation_strategy="steps"
--save_steps="500" 
--eval_steps="500" 
--save_total_limit="1" 
--logging_steps="100"
--weight_decay=1e-1
--do_eval=1
--do_train=1
--preprocessing_num_workers=1
"""
def match_numeric_type(s):
  try:
    return json.loads(s)
  except:
    return s

# construct the args dict from the args string
args_dict = dict()
for line in args_string.splitlines():
  line = line.replace('--','').replace(r'\\','').replace('"','').replace("'",'').strip()
  if line:
    if '=' in line:
      arg_name,arg_value = line.split('=')
      args_dict[arg_name] = arg_value
    else:
      args_dict[line] = ''

# match args to dataclasses
args_class_attrs = defaultdict(list)
for arg_name,arg_value in args_dict.items():
  for ArgClass in (ModelArguments, DataTrainingArguments, TrainingArguments):
    if arg_name in ArgClass.__annotations__:
      args_class_attrs[ArgClass].append(arg_name)

# initialzie args objects
model_args = ModelArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[ModelArguments]})
data_args = DataTrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[DataTrainingArguments]})
training_args = TrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[TrainingArguments]})

# Dataset Processing

In [ ]:
train_dataset = Dataset.from_pandas(train_metadata)
eval_dataset = Dataset.from_pandas(val_metadata)
test_dataset = Dataset.from_pandas(test_metadata)

if data_args.max_train_samples is not None:
    train_dataset = train_dataset.select(range(data_args.max_train_samples))

if data_args.max_val_samples is not None:
    eval_dataset = eval_dataset.select(range(data_args.max_val_samples))

# Preprocessing the datasets.
# We need to read the aduio files as arrays and tokenize the targets.
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    32000: torchaudio.transforms.Resample(32000, 16000),
}

labels = {
    bahr: bahr_index
    for bahr_index, bahr in enumerate(sorted(set(train_metadata["Bahr"])))
}
print("labels are:", labels)
print("len of them is:", len(labels))

def speech_file_to_array_fn(batch):
    start = 0
    stop = 20
    srate = 16_000
    speech_array, sampling_rate = torchaudio.load(
        f'dataset_wav/{batch["Utterance name"]}'
    )
    speech_array = speech_array[0]
    batch["speech"] = resamplers[sampling_rate](speech_array).squeeze().numpy()
    batch["sampling_rate"] = srate
    batch["parent"] = labels[batch["Bahr"]]
    return batch


labels are: {'البسيط': 0, 'الخفيف': 1, 'الرجز': 2, 'الرمل': 3, 'السريع': 4, 'الطويل': 5, 'الكامل': 6, 'المتدارك': 7, 'المتقارب': 8, 'المجتث': 9, 'المديد': 10, 'المضارع': 11, 'المقتضب': 12, 'المنسرح': 13, 'الهزج': 14, 'الوافر': 15}
len of them is: 16


In [ ]:
# for experimenting purposes
# train_dataset= Dataset.from_dict(train_dataset[:5])
# eval_dataset= Dataset.from_dict(eval_dataset[:5])
# test_dataset= Dataset.from_dict(test_dataset[:5])

In [ ]:
print("processing the training set:")
train_dataset = train_dataset.map(
    speech_file_to_array_fn,
    remove_columns=train_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

processing the training set:


  0%|          | 0/2978 [00:00<?, ?ex/s]

In [ ]:
print("Process the eval set")
eval_dataset = eval_dataset.map(
    speech_file_to_array_fn,
    remove_columns=eval_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

Process the eval set


  0%|          | 0/331 [00:00<?, ?ex/s]

In [ ]:
print("process the test set")
test_dataset = test_dataset.map(
    speech_file_to_array_fn,
    remove_columns=test_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,

)

process the test set


  0%|          | 0/359 [00:00<?, ?ex/s]

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor()
processor = CustomWav2Vec2Processor(feature_extractor=feature_extractor)

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    batch["input_values"] = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"][0],
    ).input_values
    batch["labels"] = batch["parent"]
    return batch


In [ ]:
print("Process the trainset")
train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

Process the trainset


  0%|          | 0/1489 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [ ]:
print("Process the eval set")
eval_dataset = eval_dataset.map(
    prepare_dataset,
    remove_columns=eval_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

Process the eval set


  0%|          | 0/166 [00:00<?, ?ba/s]

In [ ]:
print("Process the test set")
test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

Process the test set


  0%|          | 0/180 [00:00<?, ?ba/s]

# Prepare the model for training

In [ ]:
model = Wav2Vec2ClassificationModel.from_pretrained(
    "bakrianoo/sinai-voice-ar-stt",
    attention_dropout=0.01,
    hidden_dropout=0.01,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.01,
    # gradient_checkpointing=True,
    num_attention_heads=4,
)

if model_args.freeze_feature_extractor:
    model.freeze_feature_extractor()

def compute_metrics(pred):
    labels = pred.label_ids.argmax(-1)
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds)
    matrix = confusion_matrix(labels, preds)
    print(matrix)
    print('clasification report',report)
    print('accuracy',acc)
    return {"accuracy": acc}

# Data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# Initialize our Trainer
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=processor.feature_extractor,
)

Downloading:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at bakrianoo/sinai-voice-ar-stt were not used when initializing Wav2Vec2ClassificationModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ClassificationModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ClassificationModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ClassificationModel were not initialized from the model checkpoint at bakrianoo/sinai-voice-ar-stt and are newly initialized: ['BiGRU.weight_ih_l0_reverse', 'BiGRU.weight_hh_l3_reverse', 'BiGRU.bias_hh_l0', 'BiGRU.weight_ih_l2', 'BiGRU.weight_ih_l1_reverse', 'BiGRU.weight_ih_l2_reverse', 'BiGRU.weight_ih_l1', 'BiGRU.bias

# Train the model

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if (
    os.path.isdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        print(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

In [ ]:
# Training
if training_args.do_train:
  if last_checkpoint is not None:
      checkpoint = last_checkpoint
  elif os.path.isdir(model_args.model_name_or_path):
      checkpoint = model_args.model_name_or_path
  else:
      checkpoint = None
  train_result = trainer.train(resume_from_checkpoint=checkpoint)
  trainer.save_model()

  # save the feature_extractor and the tokenizer
  if is_main_process(training_args.local_rank):
      processor.save_pretrained(training_args.output_dir)

  metrics = train_result.metrics
  max_train_samples = (
      data_args.max_train_samples
      if data_args.max_train_samples is not None
      else len(train_dataset)
  )
  metrics["train_samples"] = min(max_train_samples, len(train_dataset))

  trainer.log_metrics("train", metrics)
  trainer.save_metrics("train", metrics)
  trainer.save_state()

# Evaluation
results = {}
if training_args.do_eval:
  print("*** Evaluate ***")
  metrics = trainer.evaluate()
  max_val_samples = (
      data_args.max_val_samples
      if data_args.max_val_samples is not None
      else len(eval_dataset)
  )
  metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

  trainer.log_metrics("eval", metrics)
  trainer.save_metrics("eval", metrics)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2978
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 7445


Step,Training Loss,Validation Loss,Accuracy
150,2.492300,2.227666,0.368580
300,2.369900,2.255484,0.329305
450,2.117000,1.879371,0.392749
600,1.958800,1.828835,0.407855
750,1.926900,1.719985,0.429003
900,1.926800,1.679819,0.447130
1050,1.687000,1.608667,0.459215
1200,1.805800,1.586779,0.477341
1350,1.665800,1.496174,0.519637
1500,1.629000,1.648785,0.447130


***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


[[  0   2   0   0   0   8   0   0   0   0   0   0   0   0   0   0]
 [  0  12   0   0   1   4   0   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0   0   0   0   0   0   0   0]
 [  0  18   0   0   5  11   0   0   0   0   0   0   0   0   0   0]
 [  0  11   0   0   5   0   0   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0   0 105   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  28   0   0   0   0   0   0   0   0   0   0]
 [  0   6   0   0   4   2   0   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   0   2   0   0   0   0   0   0   0   0   0   0]
 [  0   6   0   0   2   0   0   0   0   0   0   0   0   0   0   0]
 [  0   6   0   0   6   0   0   0   0   0   0   0   0   0   0   0]
 [  0   8   0   0   0  10   0   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  17   0   0   6   4   0   0   0   0   0   0   0   0   0   0]
 [  0   7   0   0   0   0   0   0   0   0   0   0   0   0   0 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 331
  Batch size = 

...

[[  0   0   0   0   0   1   9   0   0   0   0   0   0   0   0   0]
 [  0   5   0   0   3   2   0   1   0   1   0   0   0   5   0   0]
 [  0   0   0   0   1   0   0   0   1   0   0   0   0   0   0   0]
 [  0   0   0  34   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  14   0   0   0   0   0   0   0   0   1   1   0]
 [  0   0   0   1   0 102   4   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   4  24   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  11   0   0   0   0   0   0   1   0]
 [  0   0   0   0   0   1   2   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   2   0   0   0   0   5   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0  16   0   0   1   0]
 [  0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0]
 [  0   0   0   0   1   1   0   0   0   0   0   0   0  25   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0   0   6 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training completed. Do not forget to share your model on huggingf

***** train metrics *****
  epoch                    =          5.0
  total_flos               = 8545799613GF
  train_loss               =       1.1126
  train_runtime            =   6:11:23.86
  train_samples            =         2978
  train_samples_per_second =        0.668
  train_steps_per_second   =        0.334
*** Evaluate ***


[[  0   0   0   0   0   1   9   0   0   0   0   0   0   0   0   0]
 [  0   5   0   0   3   2   0   1   0   1   0   0   0   5   0   0]
 [  0   0   0   0   1   0   0   0   1   0   0   0   0   0   0   0]
 [  0   0   0  34   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0  13   0   0   0   0   0   0   0   0   1   1   0]
 [  0   0   0   1   0 102   4   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   3  25   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  11   0   0   0   0   0   0   1   0]
 [  0   0   0   0   0   1   2   0   0   0   0   0   0   0   0   0]
 [  0   1   0   0   2   0   0   0   0   5   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0  16   0   0   1   0]
 [  0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0]
 [  0   0   0   0   1   1   0   0   0   0   0   0   0  25   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0   0   6 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Evaluate the model

In [31]:
from sklearn.metrics import classification_report,confusion_matrix
preds = trainer.predict(test_dataset=test_dataset)
preds

***** Running Prediction *****
  Num examples = 359
  Batch size = 8


[[ 0  0  0  0  0 21  1  0  0  0  0  0  0  0  0  3]
 [ 0  9  0  0  0  0 12  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  3  0  1  0  0  0  0  0  0  0 12  0]
 [ 0  0  0  0  0 12 13  0  0  0  0  0  0  0  0  5]
 [ 0  0  0  2  0  2 14  0  0  0  0  0  0  2  0  0]
 [ 0  0  0  0  0 35  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 27  0  0]
 [ 0  0  0  0  0  0  0 16  0  0  1  0  0  0  1  0]
 [ 0  0  0  1  0  9  9  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  1  0  0  0  0 16  0  0  0  1  0  0]
 [ 0 18  0  0  2  0  0  0  0  7  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  3  0  0  0 15  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 15  0  0  0  0  0  0]
 [ 0  0  0  1  0 19  6  0  0  0  0  0  0  2  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0  0  0  0 13  0]
 [ 0  0  0  0  0 16  5  0  0  0  0  0  0  0  0  0]]
clasification report               precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.30      0.43      0.35        21

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[-1.161124  ,  3.907081  , -0.90446764, ...,  2.7890072 ,
         0.06631256, -0.54497385],
       [-0.09069643, -0.02254812,  0.6457835 , ..., -1.0836989 ,
        -1.0466216 , -0.06927547],
       [-1.0502087 ,  4.1868687 , -1.0268747 , ...,  3.3647504 ,
         0.10215414, -1.1581588 ],
       ...,
       [ 2.7655034 , -1.0100204 , -0.23350687, ...,  0.94565034,
        -2.2599967 ,  0.71215284],
       [ 0.97873235, -1.7890521 , -1.0252712 , ..., -0.35232654,
        -2.4361835 ,  0.36095393],
       [ 2.6860213 , -0.12526777, -0.13723099, ...,  3.0236688 ,
        -1.4047899 ,  0.26445165]], dtype=float32), label_ids=array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), metrics={'test_loss': 3.977369785308838, 'test_accuracy': 0.29526462395543174, 'test_runtime': 137.8722, 'test_samples_per_sec

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
# drive.mount('/content/drive',force_remount=True)